### importing packages and libraries

In [1]:
import requests
from bs4 import BeautifulSoup
import csv
import json
import xml
import pandas as pd
import numpy as np

!conda install -c conda-forge folium=0.5.0 --yes
import folium
from folium import plugins
import json
from pprint import pprint

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    altair-3.2.0               |           py36_0         770 KB  conda-forge
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.3 MB

The following NEW packages will be 

### since there isn't a decent database of Oporto localities on the internet (neither in Wikipedia), i made a dataframe with all the localities of the city of Oporto.

In [2]:
# The code was removed by Watson Studio for sharing.

,Localidade
0,Aldoar
1,Bonfim
2,Campanhã
3,Cedofeita
4,Foz do Douro
5,Lordelo do Ouro
6,Massarelos
7,Miragaia
8,Nevogilde
9,Paranhos


### getting the coordinates of Oporto localities

In [3]:
def get_coords_local(Localidade, output_as='center'):
    """
    get the bounding box of a locality in WGS84 given its name

    Parameters
    ----------
    localidad : str
        name of the country in english and lowercase
    output_as : 'str
        chose from 'boundingbox' or 'center'. 
         - 'boundingbox' for [latmin, latmax, lonmin, lonmax]
         - 'center' for [latcenter, loncenter]

    Returns
    -------
    output : list
        list with coordinates as str
    """
    # create url
    url = '{0}{1}{2}'.format('http://nominatim.openstreetmap.org/search.php?q=',
                             Localidade+', Porto, Área Metropolitana do Porto, Norte, Portugal',
                             '&format=json&polygon=0')
    response = requests.get(url).json()[0]

    # parse response to list
    if output_as == 'boundingbox':
        lst = response[output_as]
        output = [float(i) for i in lst]
    if output_as == 'center':
        lst = [response.get(key) for key in ['lat','lon']]
        output = [float(i) for i in lst]
    return output

In [4]:
df2 = df1.copy()

latitudeCln = []
longitudeCln = []
for index, row in df2.iterrows():
    print(row[0])
    lat, long = get_coords_local(Localidade=row[0], output_as='center')
    latitudeCln.append(lat)
    longitudeCln.append(long)

df2['Latitude'] = latitudeCln
df2['Longitude'] = longitudeCln

df2.shape

Aldoar
Bonfim
Campanhã
Cedofeita
Foz do Douro
Lordelo do Ouro
Massarelos
Miragaia
Nevogilde
Paranhos
Ramalde
Santo Ildefonso
São Nicolau
Sé
Vitória


(15, 3)

### Oporto localities with coordinates

In [5]:
df2

,Localidade,Latitude,Longitude
0,Aldoar,41.167935,-8.668306
1,Bonfim,41.152711,-8.597608
2,Campanhã,41.159674,-8.576985
3,Cedofeita,41.158350,-8.621314
4,Foz do Douro,41.151305,-8.670185
5,Lordelo do Ouro,41.154044,-8.651083
6,Massarelos,41.149678,-8.631936
7,Miragaia,41.145485,-8.620929
8,Nevogilde,41.164363,-8.681921
9,Paranhos,41.173513,-8.605900


In [6]:
address = 'Porto, Portugal'

geolocator = Nominatim(user_agent="capstoneProject")
location = geolocator.geocode(address, timeout=60, exactly_one=True)
latitude = location.latitude
longitude = location.longitude
print('The decimal coordinates of Porto are {}, {}.'.format(latitude, longitude))

The decimal coordinates of Porto are 41.1494512, -8.6107884.


### city map of Oporto with localities marked. 

In [7]:
# create map of Porto using latitude and longitude values
map_porto = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, local in zip(df2['Latitude'], df2['Longitude'], df2['Localidade']):
    label = '{}'.format(local)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_porto)  
    
map_porto

### using Foursquare API

In [8]:
# function to repeat the exploring process to all the localities of Porto
import urllib
def getNearbyVenues(names, latitudes, longitudes, radius=5000, categoryIds=''):
    try:
        venues_list=[]
        for name, lat, lng in zip(names, latitudes, longitudes):
            #print(name)

            # create the API request URL
            url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)

            if (categoryIds != ''):
                url = url + '&categoryId={}'
                url = url.format(categoryIds)

            # make the GET request
            response = requests.get(url).json()
            results = response["response"]['venues']

            # return only relevant information for each nearby venue
            for v in results:
                success = False
                try:
                    category = v['categories'][0]['name']
                    success = True
                except:
                    pass

                if success:
                    venues_list.append([(
                        name, 
                        lat, 
                        lng, 
                        v['name'], 
                        v['location']['lat'], 
                        v['location']['lng'],
                        v['categories'][0]['name']
                    )])

        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        nearby_venues.columns = ['Localidade', 
                  'Localidade Latitude', 
                  'Localidade Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    except:
        print(url)
        print(response)
        print(results)
        print(nearby_venues)

    return(nearby_venues)

In [9]:
LIMIT = 500 # limit of number of venues returned by Foursquare API
radius = 5000 # define radius
CLIENT_ID = 'U2OL5JWGRCHGEPUKFAVNYXDGKEOVNNAMRLKBX3RJIA4TKZQV'
CLIENT_SECRET = 'DD0TPWY3X4M4A5TQTVRKGXKVXJRE3FRZGIKF4NGG0B4UTU1I'
VERSION = '20191020'

### Oporto list of hotels by localities

In [10]:
# Use category id 4bf58dd8d48988d16c941735 to only get the burger joints
porto_venues_hotel = getNearbyVenues(names=df2['Localidade'], latitudes=df2['Latitude'], longitudes=df2['Longitude'], radius=1000, categoryIds='4bf58dd8d48988d1fa931735')
porto_venues_hotel.head()

,Localidade,Localidade Latitude,Localidade Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bonfim,41.152711,-8.597608,Hotel Nave,41.152248,-8.597209,Hotel
1,Bonfim,41.152711,-8.597608,acta the avenue,41.152275,-8.596676,Hotel
2,Bonfim,41.152711,-8.597608,Hotel Vila Galé Porto,41.150810,-8.598761,Hotel
3,Bonfim,41.152711,-8.597608,Holiday Inn Express Porto City Centre,41.152202,-8.605305,Hotel
4,Bonfim,41.152711,-8.597608,Majumica - Alojamento Local,41.150150,-8.600804,Vacation Rental


In [11]:
porto_venues_hotel.shape

(448, 7)

In [12]:
# function to add markers for given venues to map
def addToMap(df, color, existingMap):
    for lat, lng, local, venue, venueCat in zip(df['Venue Latitude'], df['Venue Longitude'], df['Localidade'], df['Venue'], df['Venue Category']):
        label = '{} ({}) - {}'.format(venue, venueCat, local)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7).add_to(existingMap)

### Oporto city map with hotels marked.

In [13]:
map_porto_hotel = folium.Map(location=[latitude, longitude], zoom_start=12)
addToMap(porto_venues_hotel, 'red', map_porto_hotel)
map_porto_hotel

### Oporto list of restaurants by localities

In [14]:
porto_venues_restaurants = getNearbyVenues(names=df2['Localidade'], latitudes=df2['Latitude'], longitudes=df2['Longitude'], radius=1000, categoryIds='4d4b7105d754a06374d81259')
porto_venues_restaurants.head()

,Localidade,Localidade Latitude,Localidade Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Aldoar,41.167935,-8.668306,Canela Moscada,41.165003,-8.670708,Bakery
1,Aldoar,41.167935,-8.668306,Soundwich,41.170314,-8.671642,Sandwich Place
2,Aldoar,41.167935,-8.668306,Hong Sha Long - wok & sushi,41.169841,-8.669774,Asian Restaurant
3,Aldoar,41.167935,-8.668306,Continente Bom Dia,41.167546,-8.654781,Supermarket
4,Aldoar,41.167935,-8.668306,Hazal,41.169876,-8.669673,Café


### Oporto city map with restaurants marked.

In [15]:
map_porto_restaurants = folium.Map(location=[latitude, longitude], zoom_start=12)
addToMap(porto_venues_restaurants, 'yellow', map_porto_restaurants)
map_porto_restaurants

### Oporto list of Nocturn life places by localities

In [16]:
porto_venues_nocturnlife = getNearbyVenues(names=df2['Localidade'], latitudes=df2['Latitude'], longitudes=df2['Longitude'], radius=1000, categoryIds='4d4b7105d754a06376d81259')
porto_venues_nocturnlife.head()

,Localidade,Localidade Latitude,Localidade Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Aldoar,41.167935,-8.668306,My Palace,41.165077,-8.672653,Brewery
1,Aldoar,41.167935,-8.668306,Boîte,41.160480,-8.672922,Nightclub
2,Aldoar,41.167935,-8.668306,InFashion Café,41.163111,-8.661400,Bar
3,Aldoar,41.167935,-8.668306,Long Drink Boavista,41.163767,-8.667509,Lounge
4,Aldoar,41.167935,-8.668306,Buffet Caplico,41.163726,-8.662688,Brewery


### Oporto city map with nocturn life places marked.

In [17]:
map_porto_nocturnlife = folium.Map(location=[latitude, longitude], zoom_start=12)
addToMap(porto_venues_nocturnlife, 'purple', map_porto_nocturnlife)
map_porto_nocturnlife

### Oporto list of subway stations by localities

In [18]:
porto_venues_subway = getNearbyVenues(names=df2['Localidade'], latitudes=df2['Latitude'], longitudes=df2['Longitude'], radius=1000, categoryIds= '4bf58dd8d48988d1fd931735')
porto_venues_subway.head()

,Localidade,Localidade Latitude,Localidade Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bonfim,41.152711,-8.597608,"Metro 24 de Agosto [A,B,C,E,F]",41.148956,-8.598372,Metro Station
1,Bonfim,41.152711,-8.597608,"Metro Bolhão [A,B,C,E,F]",41.149782,-8.605819,Metro Station
2,Bonfim,41.152711,-8.597608,Metro de S.Bento,41.143697,-8.594723,Metro Station
3,Bonfim,41.152711,-8.597608,"Metro Trindade [A,B,C,D,E,F]",41.152114,-8.609718,Metro Station
4,Bonfim,41.152711,-8.597608,"Metro Heroísmo [A,B,C,E,F]",41.146551,-8.592930,Metro Station


### Oporto city map with subway stations marked.

In [19]:
map_porto_subway = folium.Map(location=[latitude, longitude], zoom_start=12)
addToMap(porto_venues_subway, 'orange', map_porto_subway)
map_porto_subway

### Oporto list of art and leisure places by localities

In [20]:
porto_venues_art_leisure = getNearbyVenues(names=df2['Localidade'], latitudes=df2['Latitude'], longitudes=df2['Longitude'], radius=1000, categoryIds= '4d4b7104d754a06370d81259')
porto_venues_art_leisure.head()

,Localidade,Localidade Latitude,Localidade Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Aldoar,41.167935,-8.668306,Caldarium Spa,41.165401,-8.669372,General Entertainment
1,Aldoar,41.167935,-8.668306,Ovoid Garden,41.169039,-8.668710,Art Gallery
2,Aldoar,41.167935,-8.668306,Pavilhão Da Água,41.171449,-8.676532,Historic Site
3,Aldoar,41.167935,-8.668306,Monumento ao Empresário,41.162062,-8.656280,Fountain
4,Aldoar,41.167935,-8.668306,Teatro Da Vilarinha,41.172715,-8.674168,General Entertainment


### Oporto city map with art and leisure places marked 

In [21]:
map_porto_art_leisure = folium.Map(location=[latitude, longitude], zoom_start=12)
addToMap(porto_venues_art_leisure, 'green', map_porto_art_leisure)
map_porto_art_leisure

### creating a grouped dataframe with hotels, restaurants, nocturn life, subway stations and art and leisure places

In [22]:
def addColumn(startDf, columnTitle, dataDf):
    grouped = dataDf.groupby('Localidade').count()
    
    for n in startDf['Localidade']:
        try:
            startDf.loc[startDf['Localidade'] == n,columnTitle] = grouped.loc[n, 'Venue']
        except:
            startDf.loc[startDf['Localidade'] == n,columnTitle] = 0

In [23]:
df_data = df2.copy()
df_data.rename(columns={'Localidade':'Localidade'}, inplace=True)
addColumn(df_data, 'Hotel', porto_venues_hotel)
addColumn(df_data, 'Restaurants', porto_venues_restaurants)
addColumn(df_data, 'Nocturn Life', porto_venues_nocturnlife)
addColumn(df_data, 'Art and Leisure', porto_venues_art_leisure)
addColumn(df_data, 'Subway', porto_venues_subway)
df_data

,Localidade,Latitude,Longitude,Hotel,Restaurants,Nocturn Life,Art and Leisure,Subway
0,Aldoar,41.167935,-8.668306,0.0,50.0,10.0,12.0,0.0
1,Bonfim,41.152711,-8.597608,50.0,50.0,50.0,47.0,10.0
2,Campanhã,41.159674,-8.576985,2.0,50.0,12.0,14.0,4.0
3,Cedofeita,41.158350,-8.621314,50.0,50.0,49.0,47.0,5.0
4,Foz do Douro,41.151305,-8.670185,12.0,50.0,26.0,25.0,0.0
5,Lordelo do Ouro,41.154044,-8.651083,12.0,50.0,33.0,22.0,0.0
6,Massarelos,41.149678,-8.631936,49.0,50.0,50.0,48.0,1.0
7,Miragaia,41.145485,-8.620929,50.0,50.0,50.0,48.0,3.0
8,Nevogilde,41.164363,-8.681921,2.0,50.0,39.0,19.0,0.0
9,Paranhos,41.173513,-8.605900,18.0,50.0,48.0,28.0,7.0


### adding weights 

In [24]:
# negative weight: we want to open a hotel and avoid concurrence as much as possible
weight_hotel = -1

# positive weight : subway access because it makes it easy to get to the city and the airport
weight_subway = 1.5

# positive weight: restaurants, because every tourist loves portuguese food
weight_restaurants = 1.0

# positive weight: nocturn life, because tourist in portugal love Oporto's night outs.
weight_nocturnlife = 1.0

# positive weight: art and leisure, to visit beautiful places of Porto
weight_art_leisure = 1.5

In [25]:
df_weighted = df_data[['Localidade']].copy()

### final scores 

In [26]:
df_weighted['Score'] = df_data['Hotel'] * weight_hotel + df_data['Restaurants'] * weight_restaurants + df_data['Nocturn Life'] * weight_nocturnlife + df_data['Subway'] * weight_subway + df_data['Art and Leisure'] * weight_art_leisure 
df_weighted = df_weighted.sort_values(by=['Score'], ascending=False)
df_weighted

,Localidade,Score
11,Santo Ildefonso,138.5
1,Bonfim,135.5
14,Vitória,135.5
13,Sé,134.0
9,Paranhos,132.5
12,São Nicolau,129.5
3,Cedofeita,127.0
7,Miragaia,126.5
6,Massarelos,124.5
8,Nevogilde,115.5


In [28]:
map_porto_result = folium.Map(location=[latitude, longitude], zoom_start=15)

porto_win = df2[df2['Localidade'] == 'Santo Ildefonso']

for lat, lng, local in zip(porto_win['Latitude'], porto_win['Longitude'], porto_win['Localidade']):
    label = '{}'.format(local)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7).add_to(map_porto_result) 

addToMap(porto_venues_restaurants[porto_venues_restaurants['Localidade'] == 'Santo Ildefonso'], 'red', map_porto_result)
addToMap(porto_venues_hotel[porto_venues_hotel['Localidade'] == 'Santo Ildefonso'], 'green', map_porto_result)
addToMap(porto_venues_nocturnlife[porto_venues_nocturnlife['Localidade'] == 'Santo Ildefonso'], 'gold', map_porto_result)
addToMap(porto_venues_art_leisure[porto_venues_art_leisure['Localidade'] == 'Santo Ildefonso'], 'fuchsia', map_porto_result)
addToMap(porto_venues_subway[porto_venues_subway['Localidade'] == 'Santo Ildefonso'], 'orange', map_porto_result)

map_porto_result

### Santo Ildefonso is the best locality in order to open a hotel in Porto.